In [1]:
f_item = open("track1/item.txt", "r")
n = countlines(f_item)

6095

In [2]:
f_rec_log_train = open("track1/pre_rec_data.txt", "r")
m = countlines(f_rec_log_train)

15560776

In [3]:
f_user_profile = open("track1/user_profile.txt", "r")
cnt_user = countlines(f_user_profile)

2320895

In [4]:
seekstart(f_rec_log_train)
Ω = Array{Any, 1}()
for i = 1:m
    x = split(readline(f_rec_log_train), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    push!(Ω, x)
end

cnt_pos, cnt_neg = 0, 0
for x in Ω
    (x[3] == -1) && (cnt_neg += 1)
    (x[3] == 1) && (cnt_pos += 1)
end
println("$cnt_pos $cnt_neg")

3794928 11765848


In [5]:
for x in Ω[1:20]
    println(x)
end

Int32[100002, 1320812622, -1, 725379]
Int32[100002, 1320812622, -1, 1606717]
Int32[100002, 1320812622, -1, 2018059]
Int32[100002, 1320812640, 1, 1606717]
Int32[100002, 1320812640, 1, 1760376]
Int32[100002, 1320812640, 1, 1774776]
Int32[100003, 1318950627, -1, 2105515]
Int32[100003, 1318950634, -1, 725371]
Int32[100003, 1318950634, -1, 1606902]
Int32[100003, 1318950634, 1, 1774717]
Int32[100004, 1318606496, -1, 1774509]
Int32[100004, 1318606496, -1, 1774613]
Int32[100004, 1318606496, 1, 1774684]
Int32[100004, 1320747791, -1, 1606902]
Int32[100004, 1320747791, 1, 1774613]
Int32[100004, 1320747791, 1, 1774716]
Int32[100006, 1318473789, 1, 754522]
Int32[100006, 1318473789, 1, 1775025]
Int32[100006, 1318473789, 1, 2105598]
Int32[100008, 1319172149, 1, 588271]


In [6]:
struct User
    id::Int32
    age::Int16
    gender::Int8
    cnt_tweet::Int32
end

In [7]:
seekstart(f_user_profile)
U = Array{User, 1}()
for i = 1:cnt_user
    x = split(readline(f_user_profile), '\t')
    try
        x = map(t->parse(Int32, t), x[1:4])
        x[2] = (x[2] < 1950) ? 0 :
               (1950 ≤ x[2] < 2004) ? ceil((x[2] - 1950) / 2) + 1 : 
               (x[2] ≥ 2004) ? 28 : 29
    catch ArgumentError
        x[2] = "29"
        x = map(t->parse(Int32, t), x[1:4])
    end
    push!(U, User(x[1], x[2], x[3], x[4]))
end

In [8]:
user_dict = Dict{Int32, Int32}()
rev_user_dict = Dict{Int32, Int32}()
id = 0
for u in U
    id += 1
    user_dict[u.id] = id
    rev_user_dict[id] = u.id
    # (id <= 20) && println(rev_user_dict[id])
end

In [9]:
seekstart(f_item)
item_dict = Dict{Int32, Int32}()
rev_item_dict = Dict{Int32, Int32}()
for i = 1:n
    x = split(readline(f_item), '\t')
    item_dict[parse(Int32, x[1])] = i
    rev_item_dict[i] = parse(Int32, x[1])
    # (i <= 20) && println(rev_item_dict[i])
end

In [10]:
for x in Ω
    x[1], x[4] = user_dict[x[1]], item_dict[x[4]]
end

In [14]:
function evaluate(p, q, bᵤ, bᵢ)
    s = 0
    for x in Ω
        r = (x[3] == 1) ? 1 : 0
        r -= sigmoid(μ + bᵤ[x[1]] + bᵢ[x[4]] + q[:,x[4]]' * p[:,x[1]])
        s += r * r
    end
    s /= m
    s += λ₁ * sum(p .* p) + λ₂ * sum(q .* q) + λ₃ * sum(bᵤ .* bᵤ) + λ₄ * sum(bᵢ .* bᵢ)
    return s
end

function sigmoid(x)
    return 1 / (1 + exp(-x))
end

λ₁, λ₂, λ₃, λ₄ = 0.0, 0.0, 0.0, 0.0
μ = cnt_pos / m
d = 40
α = 0.001
iter, max_iter = 0, 100
#p = rand(d, cnt_user)
#q = rand(d, n)
#bᵤ = rand(1, cnt_user)
#bᵢ = rand(1, n)

lst_error = evaluate(p, q, bᵤ, bᵢ)
println("$iter $lst_error")
while true
    try
        iter += 1
        for x in Ω
            r = (x[3] == 1) ? 1 : 0
            r -= sigmoid(μ + bᵤ[x[1]] + bᵢ[x[4]] + q[:,x[4]]' * p[:,x[1]])
            bᵤ[x[1]] += α * (r - λ₁ * bᵤ[x[1]])
            bᵢ[x[4]] += α * (r - λ₂ * bᵢ[x[4]])
            q[:,x[4]] += α * (r * p[:,x[1]] - λ₃ * q[:,x[4]])
            p[:,x[1]] += α * (r * q[:,x[4]] - λ₄ * p[:,x[1]])
        end
        error = evaluate(p, q, bᵤ, bᵢ)
        (iter % 1 == 0) && println("$iter $error")
        if lst_error - error < 1e-5 #|| iter >= max_iter
            break
        end
        lst_error = error
    catch InterruptException
        break
    end
end
println(lst_error)

0 0.17913594107696626
1 0.17387649749674494
2 0.1697983188958368
3 0.16639299928111445
4 0.16348247827138676
5 0.16093689202936295
6 0.1586641501873791
7 0.15659895217624697
8 0.15469526862525124
9 0.15292062811809456
10 0.1512515094592531
11 0.14967057486825494
12 0.1481652103948581
13 0.14672654330949203
14 0.1453485861319089
15 0.14402746922007717
16 0.1427607865765109
17 0.14154704763645737
18 0.1403852358789174
19 0.1392744782385588
20 0.13821381688187054
21 0.1372020712255127
22 0.13623777762211306
23 0.13531918985746574
24 0.1344443192208312
25 0.13361099363582263
26 0.1328169207312868
27 0.1320597459579044
0.1320597459579044


In [15]:
function read_from_log_test(f)
    x = split(readline(f), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    return x
end

function test(f_rec_log_test, fout, key)
    m_test = countlines(f_rec_log_test)
    seekstart(f_rec_log_test)
    user_cnt, i = 0, 1
    cur = read_from_log_test(f_rec_log_test)
    while i <= m_test
        j = i + 1
        user_cnt += 1
        _Ω = Array{Any, 1}()
        push!(_Ω, cur)
        while j <= m_test
            cur = read_from_log_test(f_rec_log_test)
            (cur[1] != _Ω[1][1]) && break
            j += 1
            push!(_Ω, cur)
        end
        #(i ÷ 5000000 != j ÷ 5000000) && println(i, ' ', j)
        dict = Array{Any, 1}()
        for x in _Ω[1:j-i]
            x₁, x₂ = user_dict[x[1]], item_dict[x[2]]
            score = sigmoid(μ + bᵤ[x₁] + bᵢ[x₂] + q[:, x₂]' * p[:, x₁])
            push!(dict, (score, x[2]))
        end
        sort!(dict)
        write(fout, "$(_Ω[1][1]),")
        for k = 1:min(size(dict, 1), 3)
            t = size(dict, 1) - k + 1
            # (dict[t][1] < 0.5) && break
            write(fout, ((k == 1) ? "" : " ") * "$(dict[t][2])")
        end
        write(fout, "\n")
        i = j
    end
end

fout = open("test.csv", "w")
write(fout, "id,clicks\n")

# test(open("track1/rec_log_test_sorted.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pub.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pri.txt", "r"), fout, "Private")

close(fout)